#Customer Churn Anaylsis

##PDSML day 2 - importing libraries and datasets

In [ ]:
a=103
b=199
c=a+b
print(c)

###importing basic libraries

In [ ]:
import math
a=math.sqrt(245)

print(math.ceil(a))


###importing essential libs for data analysis


In [ ]:
import pandas as pd           #descriptive analysis immporting the dataset using it
import numpy as np            #mathematical functions
import matplotlib.pyplot as plt #for ploting our visuals
from sklearn.model_selection import train_test_split #this will help to slpit the data into training set and testing set
from imblearn.over_sampling import SMOTE   #synthetic minority over sampling technique #imbalance learning
import shap                                # this will help to find the factors responsible for churn





###importing dataset

In [ ]:
dataset=pd.read_csv("Telco-Customer-Churn.csv")



In [ ]:
print(f"the shape of the dataset: {dataset.shape}")

###checking value_counts()


In [ ]:
dataset['InternetService'].value_counts(normalize=True)*100

##Day3 Data cleaning

###Detecting error erroneous numeric columns by calculating the mean

In [ ]:
print(f"mean of tenure {dataset['tenure'].mean()}")


In [ ]:
print(dataset['MonthlyCharges'].mean())

###printting all the list of the blank entry in totalCharges

In [ ]:
j=0
for i in range (7042):
  if dataset['TotalCharges'].iloc[i]==' ':
    print(f"the blank is in {i+2}")
    j+=1

print(f"the count is : {j}")


###converting the blanks into nan so it can be ignored will calculating the mean

In [ ]:
dataset['TotalCharges']=pd.to_numeric(dataset['TotalCharges'],errors='coerce')
print(f"the mean: {dataset['TotalCharges'].mean()}")
dataset.shape

###it is easy to just delete the row with the nan value. so now we drop the nan containing row and check the shape of the dataset afterwards

In [ ]:
dataset=dataset.dropna(subset=['TotalCharges']).copy()
dataset.shape

##Day4 PDSML Day-4 👉 Data Preprocessing (Part-2)


###Removing unneccessary details


In [ ]:
dataset.head() #no showing all the columns


In [ ]:
list(dataset.columns) # now we can see all the columns

### Drop columns



In [ ]:
X=dataset.drop(columns=["customerID","Churn"])

X.shape

### Dependent variable Declaration


In [ ]:
print(dataset['Churn'].head())

In [ ]:
y=dataset['Churn']


### mapping the yes no to 1 0

In [ ]:
y=y.map({'No':0,'Yes':1}) #curly brackets means dictionary structure
print(y)


###Split the dataset into test set and traning set

In [ ]:
X_trainset, X_testset, y_trainset, y_testset = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)



In [ ]:
print(X_trainset.head())

In [ ]:
y.value_counts(normalize=True)


In [ ]:
y_trainset.value_counts(normalize=True)

## Day-5 👉 Data Preprocessing (Part-3)

### One hot encoding

In [ ]:
#done it using chatGpt

encoded_data=pd.get_dummies(dataset,columns=['InternetService'],dtype=int)

print(encoded_data)

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder






#### Selecting numerical columns

In [ ]:
numeric_cols=X.select_dtypes(include=np.number).columns.tolist()

print(numeric_cols)

#### Selecting the Categorical columns


In [ ]:
categorical_cols=X.select_dtypes(exclude=np.number).columns.tolist()
print(categorical_cols)

In [ ]:
pre_processed=ColumnTransformer(transformers=[
    ("cat",OneHotEncoder(handle_unknown='ignore',sparse_output=False),categorical_cols),
    ("num","passthrough",numeric_cols)
])

In [ ]:
X_preprocessed=pre_processed.fit_transform(X)
print(X_preprocessed)

## Day6 Data preprocessing (part -4)

### Getting names of encoded categorical columns

In [ ]:
cat_iv_names=pre_processed.named_transformers_['cat'].get_feature_names_out(categorical_cols)

In [ ]:
print(numeric_cols)

### Combining categorical and numerical clumns

In [ ]:
all_iv_names=list(cat_iv_names)+numeric_cols #the Cat_iv_name is not list. so need to convert it to a list

print(all_iv_names)

### Converting the transformed output to a DataFrame

In [ ]:
X_tranformed=pre_processed.fit_transform(X)


In [ ]:
print(X_tranformed)

In [ ]:
tr_df=pd.DataFrame(X_tranformed,columns=all_iv_names)
pd.set_option('display.max_columns',None)

print(tr_df)



## Day 7 - PDSML 👉 Logistic Regression Theory

### What is Simple Linear Regression?

**Definition:** Simple Linear Regression is a statistical method to model the relationship between:
- **One independent variable (X)** - predictor
- **One dependent variable (Y)** - target

**Equation:**
```
Y = β₀ + β₁X + ε
```
Where:
- **Y** = Dependent variable (target)
- **X** = Independent variable (feature)
- **β₀** = Intercept (constant)
- **β₁** = Slope (coefficient)
- **ε** = Error term

**Key Points:**
- Used for **continuous** outcomes
- Assumes linear relationship
- Best fit line minimizes errors
- Example: Predicting house price based on size

### What is Ordinary Least Squares (OLS) Method?

**Definition:** OLS is the mathematical technique used to find the best-fitting line by minimizing the sum of squared errors.

**Objective Function:**
```
Minimize: SSE = Σ(yᵢ - ŷᵢ)²
```
Where:
- **yᵢ** = Actual value
- **ŷᵢ** = Predicted value
- **Σ** = Sum of all errors squared

**How it works:**
1. Calculate predicted values using the line equation
2. Find difference between actual and predicted (residuals)
3. Square each residual (to penalize large errors)
4. Adjust the line to minimize total squared errors
5. Repeat until best fit is found

**Formulas:**
```
β₁ = Σ(xᵢ - x̄)(yᵢ - ȳ) / Σ(xᵢ - x̄)²
β₀ = ȳ - β₁x̄
```

**Advantages:**
- Simple and interpretable
- Computationally efficient
- Works well for linear relationships

**Disadvantages:**
- Sensitive to outliers
- Assumes linear relationship
- Not suitable for classification problems

### What is Multiple Linear Regression?

**Definition:** Multiple Linear Regression extends simple linear regression to use **multiple independent variables** to predict a continuous dependent variable.

**Equation:**
```
Y = β₀ + β₁X₁ + β₂X₂ + β₃X₃ + ... + βₙXₙ + ε
```

Where:
- **Y** = Dependent variable (target)
- **X₁, X₂, X₃, ..., Xₙ** = Independent variables (features)
- **β₀** = Intercept (constant)
- **β₁, β₂, β₃, ..., βₙ** = Coefficients (slopes) for each feature
- **ε** = Error term

**Example (Customer Churn Context):**
```
Churn_Score = β₀ + β₁(Tenure) + β₂(MonthlyCharges) + β₃(TotalCharges) + β₄(Age) + ε
```

**Key Points:**
- Uses **multiple features** to make predictions
- Still assumes **linear relationship**
- Better predictions than simple linear regression (usually)
- Used for **continuous outcomes**

**Advantages:**
- More flexible than simple linear regression
- Can capture complex relationships with multiple variables
- Better R² values (explained variance)

**Disadvantages:**
- **Not suitable for classification** (binary outcomes)
- Assumes linear relationships
- Prone to multicollinearity issues
- Still produces continuous output (not probabilities)

### What is Logistic Regression?

**Definition:** Logistic Regression is a classification algorithm used to predict **binary outcomes (0 or 1, Yes or No)** using a sigmoid function to convert linear regression output into probabilities.

**Key Difference from Linear Regression:**
- **Linear Regression** → Predicts continuous values
- **Logistic Regression** → Predicts **probabilities** (0 to 1) for classification

**The Sigmoid Function:**
```
σ(z) = 1 / (1 + e^(-z))
```

Where:
- **z** = Linear combination (β₀ + β₁X₁ + β₂X₂ + ... + βₙXₙ)
- **e** = Euler's number (≈ 2.718)
- **σ(z)** = Probability output (always between 0 and 1)

**Logistic Regression Equation:**
```
P(Y=1|X) = 1 / (1 + e^(-(β₀ + β₁X₁ + β₂X₂ + ... + βₙXₙ)))
```

**Interpretation:**
- Output = Probability of belonging to class 1 (Yes)
- If P > 0.5 → Classify as 1 (Churn = Yes)
- If P ≤ 0.5 → Classify as 0 (Churn = No)

**For Your Customer Churn Project:**
```
P(Churn=Yes) = 1 / (1 + e^(-(β₀ + β₁(Tenure) + β₂(MonthlyCharges) + ... )))
```

**Sigmoid Function Characteristics:**
- S-shaped curve
- Output range: 0 to 1
- At z=0, σ(z) = 0.5
- As z→∞, σ(z)→1
- As z→-∞, σ(z)→0

**Cost Function (Log Loss / Cross-Entropy):**
```
J(β) = -1/m Σ[yᵢ*log(ŷᵢ) + (1-yᵢ)*log(1-ŷᵢ)]
```

Where:
- **m** = Number of samples
- **yᵢ** = Actual value (0 or 1)
- **ŷᵢ** = Predicted probability

**Why Logistic Regression?**
1. Outputs **probabilities** (interpretable)
2. Works well for **binary classification**
3. Fast and efficient
4. Provides **feature importance** through coefficients
5. **Perfect for your churn prediction** (Churn: Yes/No)

**Advantages:**
✅ Simple and interpretable
✅ Outputs probabilities
✅ Fast training
✅ Works well with linearly separable data
✅ Provides feature coefficients (importance)
✅ Good baseline model

**Disadvantages:**
❌ Assumes linear decision boundary
❌ Struggles with non-linear patterns
❌ Sensitive to outliers
❌ Requires feature scaling for optimal performance
❌ Doesn't work well with correlated features


### Regression Models Comparison

| Aspect | Linear | Multiple Linear | Logistic |
|--------|--------|-----------------|----------|
| **Output Type** | Continuous | Continuous | Probability (0-1) |
| **Use Case** | Regression | Regression | Classification |
| **Variables** | 1 feature | Multiple features | Multiple features |
| **Equation** | Y = β₀ + β₁X | Y = β₀ + β₁X₁ + ... | P = 1/(1+e^-z) |
| **For Churn** | ❌ | ❌ | ✅ **BEST** |
| **Output Range** | -∞ to +∞ | -∞ to +∞ | 0 to 1 |
| **Decision Rule** | No threshold | No threshold | P > 0.5 = Class 1 |

## Day 8 : Training the Baseline model and evaluation

### Training the baseline model


In [38]:
from sklearn.pipeline import Pipeline # Combines multiple preprocessing and modeling steps into a single workflow
from sklearn.linear_model import LogisticRegression #Classification algorithm to predict binary outcomes (Churn: Yes/No)






In [43]:
# Create a pipeline that preprocesses features and trains a logistic regression classifier with balanced class weights
pipe_lr=Pipeline([
    ('preprocessing',pre_processed),
    ('classification_model',LogisticRegression(max_iter=5000,class_weight='balanced'))
])

pipe_lr.fit(X_trainset,y_trainset)



,steps,"[('preprocessing', ...), ('classification_model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('cat', ...), ('num', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


### Model evaluation 

#### roc_auc_score Explained
What is it?
A metric that measures how well your classification model distinguishes between two classes (0 and 1)

In [44]:
from sklearn.metrics import roc_auc_score


In [53]:
proba=pipe_lr.predict_proba(X_testset)[:,1] #that will return the proba of Churn_yes


In [ ]:
pred=(proba >= 0.5).astype(int) #uses the thres .5.. if >0.5 then it will considered as 1 (true),,0 otherwise


In [60]:
print(y_testset.head())
print(proba)


974     0
619     0
4289    0
3721    1
4533    0
Name: Churn, dtype: int64
[0.04942523 0.78696202 0.01345484 ... 0.27565946 0.06401957 0.01305002]


In [61]:
print(pred)

[False  True False ... False False False]
